In [1]:
%load_ext autoreload
%autoreload 2

import datajoint as dj

dj.config["enable_python_native_blobs"] = True
dj.config['schema_name'] = "nnfabrik_color_mei"
schema = dj.schema("nnfabrik_color_mei")
dj.config["display.limit"] = 50

from matplotlib import pyplot as plt
from torch import load

from nnfabrik.main import *
import nnfabrik
from nnfabrik import main, builder
import os
from os import listdir
from os.path import isfile, join
import torch
import matplotlib.pyplot as plt
import numpy as np
from neuralpredictors.measures import corr

from nndichromacy.tables.from_nnfabrik import TrainedModel
from nndichromacy.tables.from_mei import MEI, MEIMethod, MEISeed, MEISelector, TrainedEnsembleModel

fetch_download_path = '/data/fetched_from_attach'

Connecting kwilleke@134.2.168.16:3306


In [2]:
Dataset() & "dataset_ts > '2020-08-10'"

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,dataset_config dataset configuration object,dataset_fabrikant Name of the contributor that added this entry,dataset_comment short description,dataset_ts UTZ timestamp at time of insertion
nndichromacy.datasets.static_loaders,304aafff7cf913c98c6e9494ed3fbff9,=BLOB=,kfranke,24197-18-9 normalized,2020-08-16 06:02:27
nndichromacy.datasets.static_loaders,78aac2178ebf2d388194e55b2316e656,=BLOB=,kfranke,23656-14-22 normalized,2020-08-11 14:49:21
nndichromacy.datasets.static_loaders,9ec87ee36e77a830a15e3b9582f43760,=BLOB=,kfranke,24046-7-6 normalized,2020-08-12 08:35:52


In [3]:
key = dict(dataset_hash='304aafff7cf913c98c6e9494ed3fbff9')

In [4]:
TrainedEnsembleModel & key

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,ensemble_hash the hash of the ensemble,ensemble_comment a short comment describing the ensemble
nndichromacy.datasets.static_loaders,304aafff7cf913c98c6e9494ed3fbff9,9b3028070d7080c2e8937c87b4c4e757,default


In [5]:
dataloaders, model = (TrainedEnsembleModel & key).load_model()
model.eval();
model.cuda();

static24197-18-9-preproc0 exists already. Not unpacking static24197-18-9-preproc0.zip
static24197-18-9-preproc0 exists already. Not unpacking static24197-18-9-preproc0.zip
static24197-18-9-preproc0 exists already. Not unpacking static24197-18-9-preproc0.zip
static24197-18-9-preproc0 exists already. Not unpacking static24197-18-9-preproc0.zip
static24197-18-9-preproc0 exists already. Not unpacking static24197-18-9-preproc0.zip


In [6]:
data_key = list(dataloaders["train"].keys())[0]

In [7]:
# getting a batch of images
images, responses = next(iter(dataloaders["train"][data_key]))

In [8]:
# everything that you show to the model has to be a tensor, with 4 dimensions: batch, channels, height, width.
images.shape

torch.Size([128, 2, 36, 64])

In [9]:
# when making predictions with the model, the gradient will always be computed as well, but here it isn't needed.
# the gradient has high demands on the GPU memory, so it's best to simply do all calculations without the gradient with the following statement:

with torch.no_grad():
    output = model(images)

In [10]:
# the output is: Images x Neurons
output.shape

torch.Size([128, 1107])

In [11]:
# output of the first image and first neuron. The result is a tensor on the GPU, for further processing, it's easiest to convert to numpy
output[0,0]

tensor(0.4655, device='cuda:0')

In [12]:
output = output.cpu().numpy()
print(output[0,0])

0.46546298


# Fetching a couple of MEIs

In [22]:
n_MEIs_to_fetch = 20
mei_paths = (MEI & key).fetch("mei", limit=n_MEIs_to_fetch, download_path=fetch_download_path)

In [23]:
# loading the MEIs as GPU Tensors (to be passed to the model)
MEIs = torch.stack([torch.load(path).cuda() for path in mei_paths]).squeeze()

In [24]:
# compared to loading the MEIs as images
MEIs_images = np.stack([torch.load(path).detach().cpu().numpy().squeeze() for path in mei_paths])

In [25]:
MEIs.shape

torch.Size([20, 2, 36, 64])

In [26]:
with torch.no_grad():
    output = model(MEIs)

In [27]:
output.shape

torch.Size([20, 1107])

### The GPU should have enough memory so that you can pass all MEIs through the model at once

In [ ]:
# fetching the MEIs in the order of the unit_index (as in the model)
mei_paths = (MEISelector*MEI & key).fetch("mei", download_path=fetch_download_path, order_by="unit_index")
MEIs = torch.stack([torch.load(path).cuda() for path in mei_paths]).squeeze()

In [ ]:
with torch.no_grad():
    output = model(MEIs)

In [ ]:
# now you have the full response matrix, and from there it's easiest to convert to numpy.
# I'd guess that about 5000 images is the most the the model can take at once, without running out of memory.
# So if it's more than that, you can cut it down to batches of 5000, and show it to the model 